### test ff
Powered by [Eleonora Priori](https://www.ecoaz.unito.it/do/docenti.pl/Alias?eleonora.priori#tab-profilo) and [Pietro Terna](https://terna.to.it/) 


In [1]:
from mpi4py import MPI
from repast4py import context as ctx
import repast4py 
from repast4py import parameters
from repast4py import schedule
from repast4py import core
from typing import Tuple, List, Dict
import numpy as np
import pandas as pd
import pickle
import csv
import os
import sys
import time
import sectors

comm = MPI.COMM_WORLD
rank    = comm.Get_rank()
rankNum = comm.Get_size() 

# create the context to hold the agents and manage cross process
# synchronization
context = ctx.SharedContext(comm)

# Initialize the default schedule runner, HERE to create the t() function,
# returning the tick value
runner = schedule.init_schedule_runner(comm)



#generate random seed
repast4py.random.init(rng_seed=12345)
rng = repast4py.random.default_rng 

#timer T()
startTime=-1
def T():
    global startTime
    if startTime < 0:
        startTime=time.time()
    return time.time() - startTime
T() #launches the timer

#cpuTimer Tc()
startCpuTime=-1
def Tc():
    global startCpuTime
    if startCpuTime < 0:
        startCpuTime=time.process_time()
    return time.process_time() - startCpuTime
Tc() #launches the cpu timer


8.79999999980896e-06

In [2]:
class Firm(core.Agent):

    TYPE = 0
    
    def __init__(self, local_id: int, rank: int, sector: int, labor: int, capital: float, capitalR: float, wage: float,\
                 intermediate: list): 
                 #, minOrderDuration:int,\
                 #maxOrderDuration:int, recipe: float, laborProductivity: float, maxOrderProduction: float,\
                 #assetsUsefulLife: float, plannedMarkup: float, orderObservationFrequency: int, productionType: int,\
                 #sectorialClass: int):
        super().__init__(id=local_id, type=Firm.TYPE, rank=rank) #uid

        self.sector=sector
        self.labor=labor
        self.capital=capital
        self.capitalR=capitalR
        self.wage=wage
        self.intermediate = intermediate
        #self.investment = investment
    
        """
        self.unavailableLabor=0
        self.unavailableCapital=0
        self.minOrderDuration=minOrderDuration
        self.maxOrderDuration=maxOrderDuration
        self.recipe = recipe
        self.laborProductivity=laborProductivity
        self.maxOrderProduction=maxOrderProduction
        self.assetsUsefulLife=assetsUsefulLife
        self.plannedMarkup=plannedMarkup
        self.orderObservationFrequency=orderObservationFrequency
        self.productionType=productionType
        self.sectorialClass=sectorialClass 
        """




class Model:
    
    def __init__(self):
        
      
        runner.schedule_repeating_event(0.0,  1, self.interactingWithFirms)

        runner.schedule_stop(0)
        runner.schedule_end_event(self.finish)
        
        ####################################################################################################
        ###################################### CREATE FIRM AGENTS ##########################################
        ####################################################################################################

        
        with open('naio_io_N.xp', 'rb') as f:
            intermediateInputs = pickle.load(f) #numpy array, only numeric, see last row of firm-features-generation.ipynb
                                                #eliminating nan in col 43 (used), not in 64 (never used)
            
        #firm investment shares
        nama = 2
        while nama != 0 and nama != 1:
            nama = int(input("use nama based investment shares? (Please, use only 1/0 to express yes/no values) "))
            if nama != 0 and nama != 1: print("Please, use only 1/0 to express yes/no values")

        if nama==0: self.invShares = pd.read_pickle("./invTableNoNama.xp") #ignoring nama infos
        if nama==1: self.invShares = pd.read_pickle("./invTableNama.xp")   #using nama infos

        #65 rows x 66 columns
        #print (self.invShares)
        #print (self.invShares.iloc[0,0]) #Crop
        #print (self.invShares.iloc[1,0]) #Forest
        #print (self.invShares.iloc[0,1]) #0.002242
        #print (self.invShares.iloc[1,1]) #0.002242
        #print (self.invShares.iloc[0,2]) #0.000131
        #print (self.invShares.iloc[1,2]) #0.000131
        #print (self.invShares.iloc[43,0]) #Imputed
        #print (self.invShares.iloc[44,0]) #Real
            
        for i in range(64): #64, as rows 0:63
            intermediateInputs[i,43]=0
            #intermediateInputs[i,64]=0 #col 64 never user, as sector 65 was dropped

        fileName = "ff_with_class_limits.csv" #input("file name? ")
        simulationFirmsExAnteNumber = int(input("how many firms? "))
        
        self.smart_capital = 2
        while self.smart_capital != 0 and self.smart_capital != 1:
            self.smart_capital = int(input("use smart capital? (Please, use only 1/0 to express True/False values) "))
            if self.smart_capital != 0 and self.smart_capital != 1: print("Please, use only 1/0 to express True/False values")
                   
        
        #importing csv file containing info about firms 
        with open(fileName, newline='') as csvfile:
            firmReader= csv.reader(csvfile, delimiter=',')#, quoting=csv.QUOTE_NONNUMERIC)
            
            self.rowNumber=-1 #to skip the row of the headers
            k=0
            #for each record in .csv
            
            for row in firmReader:
                #print(row)
                labor = 0 
                if self.rowNumber>=0:
                        
                    sector = row[0]
                        
                    if row[4]=='': row[4]=0 # it pertains with last rows (naio sector 65)
                    for i in range(int(float(row[4]) * simulationFirmsExAnteNumber)// rankNum):
                        if row[0] == '1' or row[0] == '2' or row[0] == '3': #agri, silvi and fishing
                            randomizer = rng.uniform()
                            if randomizer <= 0.2: labor = 0
                            elif randomizer <= 0.9 : labor = 1
                            else: labor = 2
                        else:
                            ##if int(row[7]) == 0: row[7] = '1' #to avoid firms with 0 workers in the 0-9 class
                            labor= rng.integers(int(row[7]), int(row[8])+1) #ptptpt because integers exclude extremes                         
                            # at https://numpy.org/doc/stable/reference/random/generated/numpy.random.Generator.integers.html
                            # random.Generator.integers(low, high=None,...
                            # high, if provided, one above the largest (signed) integer to be drawn from the distribution
                            
                        capital= float(row[11]) + rng.random()*(float(row[12]) - float(row[11]))
                        if self.smart_capital:
                            capital = labor * float(row[10]) # row[10] = recipe 
                            
                        capitalR = float(row[13])
                        wage = float(row[14])

                        intermediate=[]
                        
                        for s in range(65):
                            intermediate.append(intermediateInputs[s,int(row[0])-1]*(0.95+0.10*rng.random()))
                        
                        """
                        minOrderDuration= row[5]
                        maxOrderDuration= row[6]
                        recipe= row[7] #K/L 
                        laborProductivity= row[8]
                        maxOrderProduction= row[9]
                        avgAssetsUsefulLife=row[10]  #https://www.oecd.org/sdd/productivity-stats/43734711.pdf
                        plannedMarkup=row[11]
                        orderObservationFrequency=rng.integers(row[12], row[13]+1)
                        productionType=int(row[14]) #productionType in firm-features.csv indicates the production of
                                                #investment goods if it is into the investmentGoods list in yaml
                        sectorialClass=int(self.rowNumber)
                        """
                        
                        aFirm =Firm(k, rank, int(sector), labor, capital, capitalR, wage, intermediate) #, minOrderDuration, maxOrderDuration, recipe, laborProductivity,\
                                #maxOrderProduction, avgAssetsUsefulLife, plannedMarkup, orderObservationFrequency, productionType,\
                                #sectorialClass)
                        context.add(aFirm)
                        k += 1 # first element of the UID of the agents
                self.rowNumber += 1
                self.simulationFirmsExPostNumber=k #one more, here is a count, not an id
        
    #interactingWithFirms
    def interactingWithFirms(self):

        #print(self.firmCount)

        if self.simulationFirmsExPostNumber==0:
            print("No firms created.")
        else:
            # a check
            countActualExPostFirmNumber = len(list(context.agents(agent_type=0)))
            if countActualExPostFirmNumber != self.simulationFirmsExPostNumber:
                print("DISASTER")
                quit()

            AFF_FirmNumber = 10000000
            Eu_FirmNumber = 21831369
            AFF_WorkerNumber = 9000000
            Eu_WorkerNumber = 152702115+9000000
            Eu_EmployeeCompensations = 7447036.79
            Eu_GDP_withVAT2022 = 16144780 #https://ec.europa.eu/eurostat/databrowser/view/tec00001/default/table?lang=en
            Eu_AddedValue2022 = 14303899 #naio table 2022 (milions) total Added value, gross
            Eu_Intermediate2022 = 16939701.18 	 
            propFactor = Eu_FirmNumber / countActualExPostFirmNumber

            countWorkers = 0
            countEmployeeCompensations = 0
            countCapitalR = 0
            countAFF_Firms = 0
            countAFF_Workers = 0
            intermediateTotal = 0
            intBySectors=[0]*64
            addValBySectors=[0]*64
            substitutionRate =0.07
            substitutionRateL=0.015 #for sectors 44 and 45
            firmSubstitutions=0
            #allFirmSubstitutionsByVendorSector=[0]*65
            allFirmSubstitutionsByVendorSector=np.zeros(65)
            
            for aFirm in context.agents(agent_type=0):
                countWorkers+=aFirm.labor              
                countEmployeeCompensations+=aFirm.labor*aFirm.wage
                countCapitalR+=aFirm.capital*aFirm.capitalR
                if int(aFirm.sector) <= 3: 
                    countAFF_Firms+=1
                    countAFF_Workers+=aFirm.labor
                    
                #added value
                addedValue=aFirm.labor*aFirm.wage + aFirm.capital*1000*aFirm.capitalR

                markupTentative=1.25
                addedValue*=markupTentative
                #addValBySectors[aFirm.sector-1]+=addedValue

                #intermediate goods acquisition (must consider markup)
                for s in range(64):
                    intermediateTotal += addedValue*aFirm.intermediate[s]
                    #intBySectors[aFirm.sector-1]+= addedValue*aFirm.intermediate[s]

                #capital substitutions (firm capital is in thousands of euros)
                if aFirm.sector == 43 or aFirm.sector == 44: 
                    firmSubstitutions+=aFirm.capital*substitutionRateL
                    for s in range(1,66):
                        allFirmSubstitutionsByVendorSector[s-1]+=aFirm.capital*substitutionRateL*self.invShares.iloc[aFirm.sector-1,s]
                else:                                        
                    firmSubstitutions+=aFirm.capital*substitutionRate
                    for s in range(1,66):
                        allFirmSubstitutionsByVendorSector[s-1]+=aFirm.capital*substitutionRate*self.invShares.iloc[aFirm.sector-1,s]

            print()
            table_data = [
                [' ',          'EU 27', 'simulation', 'simulation to'],
                [' ',          ' ',     ' ',          'EU scale'],
                ['AFF Firm Number', AFF_FirmNumber, countAFF_Firms, int(countAFF_Firms*propFactor)], 
                ['AFF Worker Number', AFF_WorkerNumber, countAFF_Workers, int(countAFF_Workers*propFactor)], 
                ['Firm Number', Eu_FirmNumber, countActualExPostFirmNumber, int(countActualExPostFirmNumber*propFactor)],
                ['Worker Number', Eu_WorkerNumber, countWorkers, int(countWorkers*propFactor)],
                ['Empl. Compens.', int(Eu_EmployeeCompensations), int(countEmployeeCompensations/1000000), 
                                                                int(countEmployeeCompensations*propFactor/1000000)],                                                                                                                              
                ['Capital R', int(Eu_AddedValue2022-Eu_EmployeeCompensations), int(countCapitalR/1000), 
                                                                int(countCapitalR*propFactor/1000)],
                ['GDP VAT', Eu_GDP_withVAT2022, 'N.B. contains VAT',' '],
                ['Add. Val. curr. p.', Eu_AddedValue2022, int(markupTentative*(countEmployeeCompensations/1000000\
                                                                 +countCapitalR/1000)),\
                                                    int(markupTentative*(countEmployeeCompensations*propFactor/1000000+\
                                                                         countCapitalR*propFactor/1000))],
                ['Int. goods curr. p.', int(Eu_Intermediate2022),int(intermediateTotal/1000000),\
                                                                 int(intermediateTotal*propFactor/1000000)]
            ]
            print(f"Markup tentative {100*(markupTentative-1)}%")
            for row in table_data:
                print("{: >20} {: >20} {: >20} {: >20}".format(*row))

            print("\n AFF => Agriculture, Forestry and Fishing\n\n")

            """
            t1=0
            t2=0
            for s in range(65):
                addValBySectors[s]*=propFactor/1000000
                intBySectors[s]*=propFactor/1000000
                t1+=addValBySectors[s]
                t2+=intBySectors[s]
                print(addValBySectors[s], intBySectors[s])
            
            print("\n",t1,t2)
            """

            #investments
            print(f"Investment regular substitution rate {substitutionRate}") 
            print(f"Investment long term substitution rate {substitutionRateL} for sectors 44 and 45")

            print()
            table2_data = [
                [' ',          'EU 27', 'simulation', 'simulation to'],
                [' ',          ' ',     ' ',          'EU scale'],
                ['Gross fixed c. formation','3320258.70',int(100*firmSubstitutions/1000)/100.0, int(100*(firmSubstitutions/1000)*\
                                                                            (Eu_FirmNumber/countActualExPostFirmNumber))/100.0]
                # /1000 to move to milions from thousands
            ]
            for row in table2_data:
                print("{: >20} {: >20} {: >20} {: >20}".format(*row))
               
            #by sectors
            print("\n\n\n")
            tot1=0
            tot2=0
            tot3=0
            print(f"{'vending sectors':<35}{'EU GrossCapitalFormation':<28}{'SIM. EU SCALE':<20}{'diff'}")
            for i in range(len(sectors.sectorNames)):
                inv=(allFirmSubstitutionsByVendorSector[i]/1000)*(Eu_FirmNumber/countActualExPostFirmNumber)
                diff=sectors.GrossCapitalFormation[i]-inv
                print(f"{sectors.sectorNames[i][0:30]:<35}{sectors.GrossCapitalFormation[i]:<33}{inv:<20.2f}{diff:.2f}")
                tot1+=sectors.GrossCapitalFormation[i]
                tot2+=inv
                tot3+=diff
            print(f"{'totals'[0:30]:<35}{tot1:<33.2f}{tot2:<20.2f}{tot3:.2f}")
    

    
    #finish
    def finish(self):
        print("\n\n")
        print(f"Gloabal time {T()}, cpu time {Tc()}")
        print("\nConcluded")
    
    def start(self):
        runner.execute()

In [3]:
def run():

    model = Model() 
    model.start()
    
run()

#file name: ff_with_class_limits.csv

use nama based investment shares? (Please, use only 1/0 to express yes/no values)  1
how many firms?  1000000
use smart capital? (Please, use only 1/0 to express True/False values)  1



Markup tentative 25.0%
                                    EU 27           simulation        simulation to
                                                                           EU scale
     AFF Firm Number             10000000               458052             10003563
   AFF Worker Number              9000000               412288              9004106
         Firm Number             21831369               999634             21831369
       Worker Number            161702115              7304981            159536125
      Empl. Compens.              7447036               343592              7503834
           Capital R              6856862               170682              3727606
             GDP VAT             16144780    N.B. contains VAT                     
  Add. Val. curr. p.             14303899               642843             14039300
 Int. goods curr. p.             16939701               761954             16640590

 AFF => Agriculture, Forestry and Fishing


Investm